# Poker Hand Optimization using Simulated Annealing

The question statement is given on the exam paper. As mentioned there, here is an image visualizing winning poker hands 

<img src="https://media.istockphoto.com/id/1303667887/vector/texas-holdem-poker-hand-rankings-combination-set-vector-green-background.jpg?s=612x612&w=0&k=20&c=Ax8V0VbLdPBuwS1Ibp7jzPtRAoLLd-kdhB3HIk0zyjw=">

# The Boltzmann prbability
$$
P = e^{-\frac{\Delta E}{T}}
$$

- Where is $e$ the exponent
- $\Delta E$ is the change in energy
- $T$ is the current temperature


In [ ]:
math.exp(delta_e / temp)

In [3]:
import random
import math

# Define Poker Hand Rankings
HAND_RANKINGS = {
    "High Card": 1,
    "One Pair": 2,
    "Two Pair": 3,
    "Three of a Kind": 4,
    "Straight": 5,
    "Flush": 6,
    "Full House": 7,
    "Four of a Kind": 8,
    "Straight Flush": 9,
    "Royal Flush": 10
}

# Create a standard deck of 52 cards
SUITS = ["Hearts", "Diamonds", "Clubs", "Spades"]
VALUES = {str(i): i for i in range(2, 11)}
VALUES.update({"Jack": 11, "Queen": 12, "King": 13, "Ace": 14})

DECK = [(value, suit) for value in VALUES.keys() for suit in SUITS]

# Function to generate a random hand
def generate_hand(deck, size=5):
    """Generates a random poker hand from the deck."""
    return random.sample(deck, size)

# Function to determine poker hand rank
def hand_rank(hand):
    """Determines the ranking of a poker hand."""
    values = sorted([VALUES[card[0]] for card in hand])  # Sort values
    suits = [card[1] for card in hand]

    is_flush = len(set(suits)) == 1
    is_straight = values == list(range(values[0], values[0] + 5))
    
    # Count occurrences of each value
    value_counts = {v: values.count(v) for v in values}
    counts = sorted(value_counts.values(), reverse=True)

    # Check for hand rankings
    if is_straight and is_flush:
        return HAND_RANKINGS["Royal Flush"] if values[-1] == 14 else HAND_RANKINGS["Straight Flush"]
    elif 4 in counts:
        return HAND_RANKINGS["Four of a Kind"]
    elif 3 in counts and 2 in counts:
        return HAND_RANKINGS["Full House"]
    elif is_flush:
        return HAND_RANKINGS["Flush"]
    elif is_straight:
        return HAND_RANKINGS["Straight"]
    elif 3 in counts:
        return HAND_RANKINGS["Three of a Kind"]
    elif counts == [2, 2, 1]:
        return HAND_RANKINGS["Two Pair"]
    elif 2 in counts:
        return HAND_RANKINGS["One Pair"]
    else:
        return HAND_RANKINGS["High Card"]

# Simulated Annealing Algorithm
def simulated_annealing(initial_hand, extra_pool, max_iterations=1000, start_temp=100, cooling_rate=0.99):
    """Optimizes a poker hand using Simulated Annealing."""
    current_hand = initial_hand[:]
    current_score = hand_rank(current_hand)
    temp = start_temp
    
    best_hand = current_hand[:]
    best_score = current_score

    for _ in range(max_iterations):
        if temp <= 0.1:
            break  # Stop if temperature is too low

        # Generate a new hand by swapping one card with an extra pool card
        new_hand = current_hand[:]
        hand_idx = random.randint(0, 4)  # Select a random card from the hand
        pool_idx = random.randint(0, len(extra_pool) - 1)  # Select a random card from extra pool

        new_hand[hand_idx], extra_pool[pool_idx] = extra_pool[pool_idx], new_hand[hand_idx]  # Swap

        new_score = hand_rank(new_hand)

        # Accept the new hand if it's better or with a probability based on temperature
        delta = new_score - current_score
        acceptance_prob = math.exp(delta / temp) if delta < 0 else 1.0  # Probability of accepting worse hand

        if random.random() < acceptance_prob:
            current_hand = new_hand[:]
            current_score = new_score

            # Update best hand if new one is better
            if new_score > best_score:
                best_hand = new_hand[:]
                best_score = new_score

        temp *= cooling_rate  # Reduce temperature

    return best_hand, best_score

# Main Execution
if __name__ == "__main__":
    deck = DECK[:]
    random.shuffle(deck)

    # Generate Initial Hand and Extra Pool
    initial_hand = generate_hand(deck, 5)
    remaining_deck = [card for card in deck if card not in initial_hand]
    extra_pool = generate_hand(remaining_deck, 10)  # Allow for controlled modifications

    print(f"Initial Hand: {initial_hand}, Rank: {hand_rank(initial_hand)}")
    print(f"Extra Pool: {extra_pool}")

    # Run Simulated Annealing Optimization
    optimized_hand, final_rank = simulated_annealing(initial_hand, extra_pool)

    print("\nFinal Optimized Hand:")
    print(f"Hand: {optimized_hand}, Rank: {final_rank}")


Initial Hand: [('King', 'Hearts'), ('7', 'Hearts'), ('Jack', 'Hearts'), ('Ace', 'Spades'), ('9', 'Hearts')], Rank: 1
Extra Pool: [('Queen', 'Spades'), ('Ace', 'Diamonds'), ('Queen', 'Diamonds'), ('3', 'Spades'), ('3', 'Diamonds'), ('4', 'Clubs'), ('8', 'Hearts'), ('9', 'Diamonds'), ('8', 'Diamonds'), ('6', 'Hearts')]

Final Optimized Hand:
Hand: [('3', 'Spades'), ('Ace', 'Spades'), ('3', 'Diamonds'), ('3', 'Diamonds'), ('3', 'Diamonds')], Rank: 8


# Answer of 1b here

# Answer of 1c here